<a href="https://colab.research.google.com/github/gautam-pai/PyTorch-Learning/blob/main/Convolutional_Neural_Network_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
## Convert MNIST image files into tensor 4 dimensions(# of images, height, width, color channel)

transform=transforms.ToTensor()

In [4]:
## TRain data
train_data=datasets.MNIST(root='/cnn_data',train=True,download=True,transform=transform)

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.81MB/s]


Extracting /cnn_data/MNIST/raw/train-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 154kB/s]


Extracting /cnn_data/MNIST/raw/train-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]


Extracting /cnn_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /cnn_data/MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 3.99MB/s]

Extracting /cnn_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /cnn_data/MNIST/raw



In [5]:
## Test Data

test_data=datasets.MNIST(root='/cnn_data',train=False,download=True,transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [18]:
## Creating a small batch size of 10
train_loader=DataLoader(train_data,batch_size=10,shuffle=True)
test_loader=DataLoader(test_data,batch_size=10, shuffle=False)

In [19]:
# Defining a CNN model
 # 2 Convolutional layers
 # Here the first in_Channel means, we're taking one image as input, and getting 6 outputs
 # In the sencond conv, we're giving the first conv's 6 output as input and expecting 16 output( this 16 is arbitrary, as per our wish)
conv1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=3,stride=1)


In [20]:
## Grab 1 MNIST image
for i,(X_Train,y_train) in enumerate(train_data):
  break

In [24]:
X_Train.shape


torch.Size([1, 28, 28])

In [25]:
x=X_Train.view(1,1,28,28)

In [26]:
## performing the 1st conv
x=F.relu(conv1(x))

In [28]:
x.shape

torch.Size([1, 6, 26, 26])

In [29]:
## pass through the pooling layer
x=F.max_pool2d(x,2,2)

In [31]:
x.shape

torch.Size([1, 6, 13, 13])

In [33]:
# 2nd conv layer
x=F.relu(conv2(x))

In [34]:
x.shape

torch.Size([1, 16, 11, 11])

In [35]:
## 2nd pooling layer

x=F.max_pool2d(x,2,2)

In [36]:
x.shape

torch.Size([1, 16, 5, 5])

In [37]:
## Complete model
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(1,6,3,1)
    self.conv2=nn.Conv2d(6,16,3,1)
    ## Fully connected layer
    self.fc1=nn.Linear(5*5*16,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  def forward(self,X):
    X=F.relu(self.conv1(X))
    X=F.max_pool2d(X,2,2)
    # Second Pass
    X=F.relu(self.conv2(X))
    X=F.max_pool2d(X,2,2)

    ## Flatten
    X=X.view(-1,16*5*5) # -1 so that we can vary the batch size
    # Fully connected layers
    X=F.relu(self.fc1(X))
    X=F.relu(self.fc2(X))
    X=self.fc3(X)

    return F.log_softmax(X,dim=1)



In [38]:
## Create an instance of model

torch.manual_seed(41)
model=CNN()


In [39]:
model

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [40]:
## Loss Function optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
